In [ ]:
pip install flwr tensorflow web3

In [ ]:
# Instale a biblioteca web3 com: pip install web3
from web3 import Web3

import flwr as fl
from typing import Dict, Tuple
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from solcx import compile_source, install_solc


In [ ]:
# Conectar a um nó local de blockchain (ex: Ganache ou outro simulador)
blockchain_url = "http://127.0.0.1:7545"  # Altere conforme necessário
web3 = Web3(Web3.HTTPProvider(blockchain_url))

In [ ]:
# Verificar a conexão com o nó
if web3.is_connected():
    print("Conexão com blockchain bem-sucedida")
else:
    raise Exception("Erro ao conectar na blockchain")

In [ ]:
# Conta padrão e contrato fictício (simulado para registrar atualizações)
account = web3.eth.accounts[0]
web3.eth.default_account = account

In [ ]:
# Código do contrato para registrar atualizações
contract_code = """
pragma solidity ^0.8.0;

contract FederatedUpdates {
    struct Update {
        address client;
        uint256 round;
        string dataHash;
    }

    Update[] public updates;

    function recordUpdate(uint256 round, string memory dataHash) public {
        updates.push(Update(msg.sender, round, dataHash));
    }

    function getUpdatesCount() public view returns (uint256) {
        return updates.length;
    }
}
"""

In [ ]:
# Compilar e implantar o contrato (usando o web3.py e solc combinados)
install_solc(version="0.8.0")

compiled_contract = compile_source(contract_code)
contract_interface = compiled_contract["<stdin>:FederatedUpdates"]

In [ ]:
# Implantar o contrato
FederatedUpdates = web3.eth.contract(
    abi=contract_interface["abi"], bytecode=contract_interface["bin"]
)
tx_hash = FederatedUpdates.constructor().transact()
tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
contract_address = tx_receipt.contractAddress

In [ ]:
# Obter a instância do contrato
contract = web3.eth.contract(
    address=contract_address, abi=contract_interface["abi"]
)

print(f"Contrato implantado em: {contract_address}")

In [ ]:
def create_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
class FederatedClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_test, y_test, client_id):
        self.model = model
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.client_id = client_id

    def get_parameters(self):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=1, batch_size=32)

        # Registrar atualização no blockchain
        data_hash = web3.keccak(text=str(self.model.get_weights())).hex()
        contract.functions.recordUpdate(config["round"], data_hash).transact()
        print(f"Atualização registrada no blockchain: {data_hash}")

        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test)
        return loss, len(self.x_test), {"accuracy": accuracy}

In [ ]:
def start_simulation(num_clients: int):
    # Carregar o dataset MNIST
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    # Dividir os dados para os clientes
    partition_size = len(x_train) // num_clients
    clients = []
    for i in range(num_clients):
        start = i * partition_size
        end = start + partition_size
        clients.append(
            FederatedClient(
                create_model(),
                x_train[start:end],
                y_train[start:end],
                x_test,
                y_test,
                client_id=i
            )
        )

    # Estratégia de agregação do Flower
    strategy = fl.server.strategy.FedAvg()

    # Iniciar o servidor com os clientes simulados
    fl.simulation.start_simulation(
        client_fn=lambda cid: clients[int(cid)],
        num_clients=num_clients,
        client_resources={"num_cpus": 1},
        config=fl.server.ServerConfig(num_rounds=5),
        strategy=strategy,
    )

In [ ]:
# Iniciar a simulação
start_simulation(num_clients=5)